In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dataclasses import dataclass
from tqdm import tqdm

In [50]:
# @dataclass
# class GPTconfig:
    # block_size: int = 128
    # vocab_size = 50257
    # n_embd: int = 768
    # n_layer: int = 12
    # n_head: int = 12
    # dropout: float = 0.0

batch_size: int = 16
block_size: int = 32
epochs = 5000
eval_iters = 100
learning_rate = 1e-3
# vocab_size = 50257
n_embd: int = 64
n_layer: int = 4
n_head: int = 4
dropout: float = 0.0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")    

In [51]:
with open("input.txt", "r") as f:
    text = f.read()

In [52]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [53]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x = x.to(device)
    y = y.to(device)
    return x, y

In [54]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for i in range(eval_iters):
            x, y = get_batch(split)
            logits, loss = model(x, y)
            losses[i] = loss.item()
        out[split] = losses.mean().item()
    model.train()  
    return out

In [55]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x) # (B, T, head_size)
        q = self.query(x) # (B, T, head_size)
        v = self.value(x) # (B, T, head_size)

        scale_value = k.shape[-1] ** -0.5
        attention = q @ k.transpose(-2, -1) * scale_value # (B, T, T)

        masked_attention = attention.masked_fill(self.tril[:T, :T] == 0, float("-inf")) # (B, T, T)
        masked_attention = F.softmax(masked_attention, dim=-1) # (B, T, T)
        masked_attention = self.dropout(masked_attention) # (B, T, T)

        out = masked_attention @ v # (B, T, head_size)

        return out    

In [56]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_head, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_head)])
        self.proj = nn.Linear(n_head * head_size, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = torch.cat([h(x) for h in self.heads], dim=-1)
        return self.dropout(self.proj(x))


In [57]:
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(n_embd, n_embd * 4),
            nn.ReLU(), 
            nn.Linear(n_embd * 4, n_embd),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.layers(x)

In [58]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.self_attn = MultiHeadAttention(n_head, head_size)
        self.feed_forward = FeedForward(n_embd)
        self.layer_norm1 = nn.LayerNorm(n_embd)
        self.layer_norm2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.self_attn(self.layer_norm1(x))
        x = x + self.feed_forward(self.layer_norm2(x))
        return x

In [82]:
class GPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embeddings = nn.Embedding(vocab_size, n_embd)
        self.position_embeddings = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embeddings(idx) # (B, T, C)
        pos_emb = self.position_embeddings(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)

        logits = self.head(x)

        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        else:
            loss = None

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
            

In [90]:
model = GPT()
model = model.to(device)
print(sum(p.numel() for p in model.parameters()))
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


209729


  0%|                                                                      | 0/5000 [00:00<?, ?it/s]

  0%|                                                              | 5/5000 [00:01<21:41,  3.84it/s]

epoch 0, train loss 4.302, valid loss 4.271


  2%|█▎                                                          | 105/5000 [00:06<08:57,  9.11it/s]

...

epoch 4800, train loss 2.200, valid loss 2.212


 98%|█████████████████████████████████████████████████████████▊ | 4903/5000 [06:18<00:30,  3.16it/s]

epoch 4900, train loss 2.186, valid loss 2.206


100%|███████████████████████████████████████████████████████████| 5000/5000 [06:24<00:00, 13.00it/s]

epoch 4999, train loss 2.256, valid loss 2.220


In [100]:
for epoch in tqdm(range(1000), ncols=100):
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()  

    if epoch % eval_iters == 0 or epoch == epochs - 1:
        losses = estimate_loss()
        print(f'epoch {epoch}, train loss {loss.item():.3f}, valid loss {losses["val"]:.3f}')

  0%|▏                                                             | 4/1000 [00:01<05:49,  2.85it/s]

epoch 0, train loss 2.023, valid loss 2.098


 11%|██████▎                                                     | 106/1000 [00:06<01:37,  9.14it/s]

epoch 100, train loss 1.934, valid loss 2.090


 21%|████████████▎                                               | 206/1000 [00:10<01:28,  8.98it/s]

...

 81%|████████████████████████████████████████████████▎           | 806/1000 [00:38<00:25,  7.53it/s]

epoch 800, train loss 2.075, valid loss 2.078


 90%|██████████████████████████████████████████████████████▏     | 904/1000 [00:44<00:23,  4.04it/s]

epoch 900, train loss 2.042, valid loss 2.078


100%|███████████████████████████████████████████████████████████| 1000/1000 [00:48<00:00, 20.68it/s]


In [101]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=1000)[0].tolist()))


Fethy ruls, for, qut.

EY VINCER:
Pow, bemres!
Thou mure ore rump hy carept, and anlive.
As him pante thyee ofrunt. Renges:
I she il me desparse tuing have say Setrored,-
O'GLIOLIUKE-
DWHUKINA:
Til, delly go thim
Marbes camendy!
Sewor' him dond a sereiid day.
Thon your madady me thou thas toundentle.

DUCTALIUCHBES:

IGRGAO:
F deacl in, love.

KING TIIVM:
I powita wor not chis is, the! the ut I'lCYoon

FHORULAPE:

Diest u.
JULARIS:
Th wing wour that kere! Thou los this tha me turcke sorough.

POM! INGTEDW:
But It ard in that diall known sities flaqied;
Aund, cope wethe and, by have
I'e are me diths manse bed
Sith, all toin cissmand, my:
sup wich caingh'd sire of
Lor thee is kis nor well.
QTON'US:
Yoave ay the meake think:
I thak
I sthereedserpad, this leese;
See with lictimenk exsgh mer bepte knot thootir. Save grent bearfulm? word,
What a and in lamefner'nt, this do tuby.
The weweld she firme lo, tho agu'g, trow rentu,--

TUCES:
Thas ha me?
What rast'd men thorthe sullfaing awit
Thee